In [5]:
import pandas as pd
from rq6.utils import RESOURCES

In [11]:
ranger_data_path = RESOURCES / "adjusted_evaluated_ranger_data.csv"
df = pd.read_csv(ranger_data_path)

In [12]:
df

,GA,version,compatible_versions,compatible_versions (ours),err
0,ch.epfl.scala:bloop-backend_2.12,1.4.11-41-810654d1,"['1.4.12-16-373bc94e', '1.4.12-14-2cef762b', '...",NaN,no_maven
1,ch.epfl.scala:bloop-backend_2.12,1.4.12-4-b38916d2,"['1.4.12-16-373bc94e', '1.4.12-14-2cef762b', '...",NaN,no_maven
2,ch.epfl.scala:bloop-frontend_2.12,1.4.11,"['1.4.12-16-373bc94e', '1.4.12-14-2cef762b', '...",NaN,no_maven
3,ch.epfl.scala:bloop-shared_2.12,1.4.11-19-93ebe2c6,"['1.4.12-16-373bc94e', '1.4.12-14-2cef762b', '...",NaN,no_maven
4,ch.epfl.scala:bloop-shared_2.12,1.4.11-28-dd429fcb,"['1.4.12-16-373bc94e', '1.4.12-14-2cef762b', '...",NaN,no_maven
...,...,...,...,...,...
475,us.ihmc:scs2-simulation,17-0.12.2,"['17-0.12.4', '17-0.12.3', '17-0.12.2']",NaN,no_github_link
476,us.ihmc:scs2-simulation,17-0.12.3,"['17-0.12.4', '17-0.12.3']",NaN,no_github_link
477,us.ihmc:scs2-simulation,17-0.8.2,"['17-0.11.1', '17-0.11.0', '17-0.10.3', '17-0....",NaN,no_github_link
478,us.ihmc:scs2-simulation,17-0.8.6-bullet-alpha-7,"['17-0.11.1', '17-0.11.0', '17-0.10.3', '17-0....",NaN,no_github_link


In [21]:
unique_gas = df.groupby(['GA']).size()
unique_gas

GA
ch.epfl.scala:bloop-backend_2.12                          2
ch.epfl.scala:bloop-frontend_2.12                         1
ch.epfl.scala:bloop-shared_2.12                          20
com.aliyun.openservices:aliyun-log-log4j2-appender        1
com.aliyun.schedulerx:schedulerx2-spring-boot-starter     1
                                                         ..
us.ihmc:scs2-session                                     18
us.ihmc:scs2-session-logger                               1
us.ihmc:scs2-session-visualizer-jfx                       3
us.ihmc:scs2-simulation                                  16
us.ihmc:scs2-simulation-construction-set                  1
Length: 157, dtype: int64

In [22]:
print(f"Number of unique GAs: {len(unique_gas)}")
print(f"Number of unique GAVs: {len(df)}")

Number of unique GAs: 157
Number of unique GAVs: 480


In [28]:
err_counts = df['err'].value_counts()
print(f"distribution of errs: ")
print(err_counts)

distribution of errs: 
err
no_github_link    368
no_maven           78
no_github_tag      18
no_test             6
no_compile          5
Name: count, dtype: int64


In [27]:
not_empty_count = df['compatible_versions (ours)'].notna().sum()
print("Number of times 'compatible_versions (ours)' column is not empty:", not_empty_count)

Number of times 'compatible_versions (ours)' column is not empty: 5


In [37]:
filtered_df = df[df['compatible_versions (ours)'].notna()]
filtered_df

,GA,version,compatible_versions,compatible_versions (ours),err
49,com.indoqa:indoqa-boot,0.12.0,"['0.16.0', '0.15.0', '0.14.0', '0.13.0', '0.12...","['0.11.0', '0.10.0', '0.13.0', '0.14.0', '0.15...",NaN
65,eu.unicore.security:securityLibrary,5.3.1,"['5.3.2', '5.3.1']","['5.3.0', '5.3.2', '5.3.3', '5.3.4', '5.3.6', ...",NaN
156,org.dhatim:dropwizard-sentry,2.0.25-2,"['2.0.26-1', '2.0.25-2']","['2.0.25-1', '2.0.25', '2.0.26-1', '2.0.26-2',...",NaN
166,org.robotframework:jrobotremoteserver,4.0.1,"['4.1.0', '4.0.1']","['4.0.0', '4.1.0', '4.0.1']",NaN
178,org.spdx:spdx-tools,2.2.5,"['2.2.6', '2.2.5']",['2.2.5'],NaN


In [68]:
# Find the intersection of sets
intersection = []
only_in_versions = []
only_in_ours = []

for index, row in filtered_df.iterrows():
    ranger_result = pd.eval(row['compatible_versions'])
    our_result = pd.eval(row['compatible_versions (ours)'])
    intersection = [x for x in ranger_result if x in our_result]
    only_ranger = [x for x in ranger_result if x not in our_result]
    only_our = [x for x in our_result if x not in ranger_result]
    print(f"GAV: {row['GA']}:{row['version']}")
    print(f"  ranger: {ranger_result}")
    print(f"  our: {our_result}")
    print(f"  intersection: {intersection}")
    print(f"  only_ranger: {only_ranger}")
    print(f"  only_our: {only_our}")
    print(f"    Our result covered {(len(intersection) / len(ranger_result) * 100)}% of ranger's result")
    print(f"    Ranger included {len(only_ranger)} versions not included in our range.")
    print(f"    Our tool included {len(only_our)} versions not included in Ranger's range.")
    if len(ranger_result) > len(our_result):
        print(f"    Ranger has the largest range, with {len(ranger_result) - len(our_result)} more versions")
    elif len(ranger_result) < len(our_result):
        print(f"    Our tool has the largest range, with {len(our_result) - len(ranger_result)} more versions")
        
    print()
    

GAV: com.indoqa:indoqa-boot:0.12.0
  ranger: ['0.16.0', '0.15.0', '0.14.0', '0.13.0', '0.12.0']
  our: ['0.11.0', '0.10.0', '0.13.0', '0.14.0', '0.15.0', '0.16.0', '0.12.0']
  intersection: ['0.16.0', '0.15.0', '0.14.0', '0.13.0', '0.12.0']
  only_ranger: []
  only_our: ['0.11.0', '0.10.0']
    Our result covered 100.0% of ranger's result
    Ranger included 0 versions not included in our range.
    Our tool included 2 versions not included in Ranger's range.
    Our tool has the largest range, with 2 more versions

GAV: eu.unicore.security:securityLibrary:5.3.1
  ranger: ['5.3.2', '5.3.1']
  our: ['5.3.0', '5.3.2', '5.3.3', '5.3.4', '5.3.6', '5.3.1']
  intersection: ['5.3.2', '5.3.1']
  only_ranger: []
  only_our: ['5.3.0', '5.3.3', '5.3.4', '5.3.6']
    Our result covered 100.0% of ranger's result
    Ranger included 0 versions not included in our range.
    Our tool included 4 versions not included in Ranger's range.
    Our tool has the largest range, with 4 more versions

GAV: org